Допустим, у нас есть $n$ товаров с заданными стоимостями $v_i$ и массой $w_i$. В сумку убирается $С$ кг. Сколько какого товара взять, чтобы сумма всех стоимостей товаров была наибольшей?

In [1]:
values = [4, 2, 1, 7, 3, 6]
weights = [5, 9, 8, 2, 6, 5]
C = 15
n = 6

Сформулируем задачу:
$$\max\sum v_i x_i$$
$$\sum w_i x_i \leq C $$

Как должна выглядеть задача:
$$\min c^T x$$
$$A x \leq b $$

Получается, что $c=-v$, $A=w^T$, $b=(C)$

In [2]:
import numpy as np

In [3]:
c = - np.array(values)
A = np.array(weights)  # shape = (6,)
A = np.expand_dims(A, 0)  # shape = (1,6)
b = np.array([C])

In [4]:
from scipy.optimize import linprog

In [5]:
linprog(c=c, A_ub=A, b_ub=b)

     con: array([], dtype=float64)
     fun: -52.50000000003077
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([-2.2495783e-11])
  status: 0
 success: True
       x: array([6.18738532e-14, 1.05853306e-12, 1.21475943e-13, 7.50000000e+00,
       4.00246692e-13, 4.71394162e-13])

In [6]:
!pip install cvxpy

In [7]:
import cvxpy

In [8]:
x = cvxpy.Variable(shape=n, integer=True)

In [9]:
constraint = (A @ x <= b)
total_value = c * x

/Users/eldarkalachev/opt/anaconda3/lib/python3.8/site-packages/cvxpy/expressions/expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


In [10]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint])

/Users/eldarkalachev/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
problem.solve(solver='ECOS_BB')

-138412039.0000002

In [12]:
x.value

array([ 8388608.00000001, -8388608.00000001, -8388608.00000001,
        8388608.00000001,  6990509.00000001,  8388608.00000001])

Теперь положительные $x$

In [13]:
x = cvxpy.Variable(shape=n, integer=True)
constraint = (A @ x <= b)
x_positive = (x >= 0)
total_value = c * x
problem = cvxpy.Problem(cvxpy.Minimize(total_value),
                        constraints=[constraint, x_positive])
round(problem.solve(solver='ECOS_BB'))

/Users/eldarkalachev/opt/anaconda3/lib/python3.8/site-packages/cvxpy/expressions/expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
/Users/eldarkalachev/opt/anaconda3/lib/python3.8/site-packages/cvxpy/problems/problem.py:1125: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


-49

In [14]:
x.value

/Users/eldarkalachev/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([7.01265807e-10, 7.99333027e-10, 3.58703130e-10, 7.00000000e+00,
       4.67143021e-10, 9.34955115e-10])

Теперь $x = 0$ или $1$

In [15]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = A @ x <= b
x_positive = x >= 0
total_value = c * x
problem = cvxpy.Problem(cvxpy.Minimize(total_value),
                        constraints=[constraint, x_positive])
round(problem.solve(solver='ECOS_BB'))

/Users/eldarkalachev/opt/anaconda3/lib/python3.8/site-packages/cvxpy/expressions/expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


-17

In [16]:
x.value

/Users/eldarkalachev/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([1.00000000e+00, 2.25474413e-10, 2.07396078e-10, 1.00000000e+00,
       2.24003299e-10, 1.00000000e+00])

# Задача: 1

Составьте оптимальный план перевозок, со Склада № 1 и Склада № 2, в три торговых центра, с учётом тарифов, запасов и потребностей, которые указаны в таблице:

![task1](./img/task1.png)

Сформулируйте задачу, как задачу линейного программирования, и решите её любым способом (желательно программным).

В ответ запишите минимальную суммарную стоимость поставки (с точностью до целых).

In [17]:
cost = np.array([
    [2, 5, 3],
    [7, 7, 6]
])

y = cvxpy.Variable(shape=(2, 3), integer=True)

constraint = [
    # суммы по рядам (ограничения, связанные с объёмом продукции на складах)
    cvxpy.sum(y[0]) <= 180,
    cvxpy.sum(y[1]) <= 220,
    # суммы по столбцам (ограничения, связанные с объёмом необходимой продукции в ТЦ)
    cvxpy.sum(y[:, 0]) == 110,
    cvxpy.sum(y[:, 1]) == 150,
    cvxpy.sum(y[:, 2]) == 140,
    # неотрицательность элементов y
    y >= 0
]

total_value = cvxpy.sum(cvxpy.multiply(cost, y))
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=constraint)

np.round(problem.solve(solver='ECOS_BB'))

1900.0

# Задача: 2

Решите задачу о назначениях

![task1](./img/task2.png)

В ответ запишите минимальную стоимость.

### Решение:
1. Составим условия:

$$min \sum_{i, j} c_{ij} x_{ij}$$

$$x_{ij} \leq 1$$

$$\forall i: \sum_{j} x_{ij} = 1$$

$$\forall j: \sum_{i} x_{ij} = 1$$

In [18]:
c = np.array([
    [1000, 12, 10, 19, 8],
    [12, 1000, 3, 7, 2],
    [10, 3, 1000, 6, 20],
    [19, 7, 6, 1000, 4],
    [8, 2, 20, 4, 1000]
])

x = cvxpy.Variable(shape=(5,5), boolean=True)

constraints = [
    # суммы по исполнителям (каждый исполнитель может выполнить максимум одну задачу)
    cvxpy.sum(x, axis=0) == np.ones(5),
    # суммы по задачам (каждая задача должна быть выполнена один раз)
    cvxpy.sum(x, axis=1) == np.ones(5)
]

total_value = cvxpy.sum(cvxpy.multiply(x, c))
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=constraints)

np.round(problem.solve(solver='ECOS_BB'))

32.0

# Задача 3:

Необходимо найти кратчайший маршрут из точки **A**, который проходит через все другие точки и возвращается в **A**.

![task1](./img/task3.png)

Сформулируйте эту задачу как задачу ЦЛП и решите её.

В ответ запишите длину кратчайшего пути.

### Решение:

$$min \sum_{i, j} d_{ij} x_{ij}$$

$$x_{ij} \leq 1$$

$$\forall i: \sum_{j} x_{ij} = 1$$

$$\forall j: \sum_{i} x_{ij} = 1$$

$$0 \leq u_{i} \leq n - 1$$

$$u_{i} - u_{j} + nx_{ij} \leq n - 1; \forall i \in \{1, ..., n\}, \forall j \in \{2, ..., n\}, 2 \leq i \neq j \leq n$$

- Заведем две переменные x и u - матрица, которая будет задавать путь, и вспомогательная переменная.
- Далее задаём ограничения (взял из предыдущего блока + добавил ограничение **cvxpy.sum(cvxpy.diag(x)) == 0** - диагональ полученной матрицы пути должна быть нулевой, чтобы исключить возможность перехода из точки в саму же себя).
- Далее d - матрица, задающая длину путей. На диагоналях (переход из точки в саму себя) проставил нули.
- Последний этап - решение с помощью функции из **cvxpy**.

In [19]:
from itertools import product

In [20]:
x = cvxpy.Variable(shape=(5, 5), boolean=True)
u = cvxpy.Variable(shape=5, integer=True)

constraints = [
    cvxpy.sum(x, axis=0) == np.ones(5),
    cvxpy.sum(x, axis=1) == np.ones(5),
    u >= 0,
    u <= 4,
    cvxpy.sum(cvxpy.diag(x)) == 0
]

for i, j in product(range(5), range(5)):
    if i >= 0 and j >= 1 and i != j:
        constraints.append(u[i] - u[j] + 5 * x[i, j] <= 4)
        
d = np.array([[0, 12, 10, 19, 8],
              [12, 0, 3, 7, 2],
              [10, 3, 0, 6, 20],
              [19, 7, 6, 0, 4],
              [8, 2, 20, 4, 0]])

func = cvxpy.sum(cvxpy.multiply(x, d))
problem = cvxpy.Problem(cvxpy.Minimize(func), constraints=constraints)
print(np.round(problem.solve(solver='ECOS_BB')))

32.0
